In [48]:
from collections import defaultdict

N, E, S, W = 0, 1, 2, 3


class Cart:
    node = None
    heading = None
    choices = 0
    active = True
    
    def __init__(self, node, heading):
        self.node = node
        self.heading = heading
        node.cart = self
    
    def move(self):
        self.heading = self.node.next_heading(self.heading, self.choices)
        next_node = self.node.links[self.heading]
        self.node.cart = None
        if next_node.cart:
            crashee = next_node.cart
            self.active = False
            crashee.active = False
            next_node.cart = None
            return crashee
        if self.node.choice:
            self.choices = (self.choices + 1) % 3
        next_node.cart = self
        self.node = next_node

        
class Node:
    cart = None
    coords = None
    links = None
    choice = False
    
    def __init__(self, r, c):
        self.coords = (c, r)
        self.links = [None] * 4

    def connect(self, left_node, top_node):
        raise NotImplememtedError
    
    def next_heading(self, heading, turns):
        raise NotImplementedError

        
class StraightNode(Node):
    def next_heading(self, heading, turns):
        return heading

    
class VertNode(StraightNode):
    def connect(self, left_node, top_node):
        self.links[N] = top_node
        top_node.links[S] = self

        
class HorNode(StraightNode):
    def connect(self, left_node, top_node):
        self.links[W] = left_node
        left_node.links[E] = self

        
class TurnNode(Node):
    # array that maps input heading (N=0, E=1, S=2, W=3) to output heading
    turns = [None, None, None, None]
    
    def next_heading(self, heading, turns):
        return self.turns[heading]

    def connect(self, left_node, top_node):
        if top_node:
            self.links[N] = top_node
            top_node.links[S] = self
        if left_node:
            self.links[W] = left_node
            left_node.links[E] = self

            
class SlashNode(TurnNode):
    turns = [E, N, W, S]

    
class BackslashNode(TurnNode):
    turns = [W, S, E, N]

    
class InterNode(Node):
    choice = True
    turns = [W, N, E, S, W, N]
    
    def next_heading(self, heading, turns):
        return self.turns[heading + turns]

    def connect(self, left_node, top_node):
        self.links[N] = top_node
        top_node.links[S] = self
        self.links[W] = left_node
        left_node.links[E] = self
        
    
class Board:
    time = None
    carts = None
    
    def __init__(self, filename):
        parse_node = {
            '<': HorNode,
            '>': HorNode,
            '^': VertNode,
            'v': VertNode,
            '|': VertNode,
            '-': HorNode,
            '/': SlashNode,
            '\\': BackslashNode,
            '+': InterNode,
            ' ': None,
            '\n': None,
        }
        parse_heading = {
            '<': W,
            '>': E,
            '^': N,
            'v': S,
        }
        carts = []
        with open(filename) as infile:
            row = defaultdict(lambda : None)
            for r, l in enumerate(infile):
                prev_row = row
                row = []
                for c, rune in enumerate(l):
                    node = None
                    node_class = parse_node[rune]
                    if node_class:
                        node = node_class(r, c)
                        try:
                            node.connect(row and row[-1], prev_row[c])
                        except:
                            print('{} {} {}'.format(rune, r, c))
                            raise
                        if rune in parse_heading:
                            carts.append(Cart(node, parse_heading[rune]))
                    row.append(node)
        self.carts = carts
        self.time = 0

    def tick(self):
        self.time += 1
        crash = None
        for cart in sorted(self.carts, key=lambda c: c.node.coords):
            if cart.active:
                crashee = cart.move()
                if crashee:
                    self.carts.remove(cart)
                    self.carts.remove(crashee)
                    crash = crash or crashee.node.coords
        return crash
        

In [49]:
b = Board('input.txt')
crash = None
while not crash:
    crash = b.tick()
crash

(83, 49)

In [50]:
b = Board('input.txt')
while(len(b.carts) > 1):
    b.tick()
b.carts.pop().node.coords

(73, 36)